In [ ]:
import wandb

In [ ]:
!wandb login


In [ ]:
import wandb
import os
os.environ['WANDB_DIR'] = os.getcwd() + '/wandb/'
os.environ['WANDB_CACHE_DIR'] = os.getcwd() + '/wandb/.cache/'
os.environ['WANDB_CONFIG_DIR'] = os.getcwd() + '/wandb/.config/'

import torch
import torch.nn as nn
import torch.optim as optim

# 初始化wandb
wandb.init(
    entity='kaifan-li',
    project="my_project"
)

# 构建模型
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.fc = nn.Linear(10, 1)
    
    def forward(self, x):
        return self.fc(x)

model = SimpleModel()

# 定义损失函数和优化器
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# 训练循环
for epoch in range(10):
    running_loss = 0.0
    for i in range(100):
        inputs = torch.randn(32, 10)  # 随机生成输入数据
        labels = torch.randn(32, 1)   # 随机生成标签
        
        # 正向传播
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    # 记录训练过程和指标
    avg_loss = running_loss / 100
    wandb.log({"epoch": epoch, "loss": avg_loss})


----

In [ ]:
long_text = "Long documents, like research papers, scientific articles, or books, often contain a wealth of information and insights. These documents can cover complex topics, present detailed arguments, and provide extensive evidence to support their claims. However, handling such long texts can be challenging, especially when working with models that have input length limitations.n natural language processing tasks, like text classification or language generation, it is common to use pretrained transformer models like BERT, GPT-3, or T5. These models typically have a maximum input length of 512 tokens, which means they cannot directly handle documents that exceed this limit.To process long documents with pretrained models, a common approach is to split the text into smaller segments or chunks, process each segment independently with the model, and then combine the results. By dividing the document into smaller parts, each segment can fit within the model's input length constraints.However, this approach requires careful handling to ensure that the divisions do not disrupt the coherence and context of the document. Some strategies involve using sliding windows, adding special tokens to mark the beginning and end of segments, or using an overlap between segments to preserve context.Researchers and developers often implement custom solutions to tackle long document processing, depending on their specific use case and model requirements.Long documents, like research papers, scientific articles, or books, often contain a wealth of information and insights. These documents can cover complex topics, present detailed arguments, and provide extensive evidence to support their claims. However, handling such long texts can be challenging, especially when working with models that have input length limitations.n natural language processing tasks, like text classification or language generation, it is common to use pretrained transformer models like BERT, GPT-3, or T5. These models typically have a maximum input length of 512 tokens, which means they cannot directly handle documents that exceed this limit.To process long documents with pretrained models, a common approach is to split the text into smaller segments or chunks, process each segment independently with the model, and then combine the results. By dividing the document into smaller parts, each segment can fit within the model's input length constraints.However, this approach requires careful handling to ensure that the divisions do not disrupt the coherence and context of the document. Some strategies involve using sliding windows, adding special tokens to mark the beginning and end of segments, or using an overlap between segments to preserve context.Researchers and developers often implement custom solutions to tackle long document processing, depending on their specific use case and model requirements.Long documents, like research papers, scientific articles, or books, often contain a wealth of information and insights. These documents can cover complex topics, present detailed arguments, and provide extensive evidence to support their claims. However, handling such long texts can be challenging, especially when working with models that have input length limitations.n natural language processing tasks, like text classification or language generation, it is common to use pretrained transformer models like BERT, GPT-3, or T5. These models typically have a maximum input length of 512 tokens, which means they cannot directly handle documents that exceed this limit.To process long documents with pretrained models, a common approach is to split the text into smaller segments or chunks, process each segment independently with the model, and then combine the results. By dividing the document into smaller parts, each segment can fit within the model's input length constraints.However, this approach requires careful handling to ensure that the divisions do not disrupt the coherence and context of the document. Some strategies involve using sliding windows, adding special tokens to mark the beginning and end of segments, or using an overlap between segments to preserve context.Researchers and developers often implement custom solutions to tackle long document processing, depending on their specific use case and model requirements.Long documents, like research papers, scientific articles, or books, often contain a wealth of information and insights. These documents can cover complex topics, present detailed arguments, and provide extensive evidence to support their claims. However, handling such long texts can be challenging, especially when working with models that have input length limitations.n natural language processing tasks, like text classification or language generation, it is common to use pretrained transformer models like BERT, GPT-3, or T5. These models typically have a maximum input length of 512 tokens, which means they cannot directly handle documents that exceed this limit.To process long documents with pretrained models, a common approach is to split the text into smaller segments or chunks, process each segment independently with the model, and then combine the results. By dividing the document into smaller parts, each segment can fit within the model's input length constraints.However, this approach requires careful handling to ensure that the divisions do not disrupt the coherence and context of the document. Some strategies involve using sliding windows, adding special tokens to mark the beginning and end of segments, or using an overlap between segments to preserve context.Researchers and developers often implement custom solutions to tackle long document processing, depending on their specific use case and model requirements.Long documents, like research papers, scientific articles, or books, often contain a wealth of information and insights. These documents can cover complex topics, present detailed arguments, and provide extensive evidence to support their claims. However, handling such long texts can be challenging, especially when working with models that have input length limitations.n natural language processing tasks, like text classification or language generation, it is common to use pretrained transformer models like BERT, GPT-3, or T5. These models typically have a maximum input length of 512 tokens, which means they cannot directly handle documents that exceed this limit.To process long documents with pretrained models, a common approach is to split the text into smaller segments or chunks, process each segment independently with the model, and then combine the results. By dividing the document into smaller parts, each segment can fit within the model's input length constraints.However, this approach requires careful handling to ensure that the divisions do not disrupt the coherence and context of the document. Some strategies involve using sliding windows, adding special tokens to mark the beginning and end of segments, or using an overlap between segments to preserve context.Researchers and developers often implement custom solutions to tackle long document processing, depending on their specific use case and model requirements.Long documents, like research papers, scientific articles, or books, often contain a wealth of information and insights. These documents can cover complex topics, present detailed arguments, and provide extensive evidence to support their claims. However, handling such long texts can be challenging, especially when working with models that have input length limitations.n natural language processing tasks, like text classification or language generation, it is common to use pretrained transformer models like BERT, GPT-3, or T5. These models typically have a maximum input length of 512 tokens, which means they cannot directly handle documents that exceed this limit.To process long documents with pretrained models, a common approach is to split the text into smaller segments or chunks, process each segment independently with the model, and then combine the results. By dividing the document into smaller parts, each segment can fit within the model's input length constraints.However, this approach requires careful handling to ensure that the divisions do not disrupt the coherence and context of the document. Some strategies involve using sliding windows, adding special tokens to mark the beginning and end of segments, or using an overlap between segments to preserve context.Researchers and developers often implement custom solutions to tackle long document processing, depending on their specific use case and model requirements.Long documents, like research papers, scientific articles, or books, often contain a wealth of information and insights. These documents can cover complex topics, present detailed arguments, and provide extensive evidence to support their claims. However, handling such long texts can be challenging, especially when working with models that have input length limitations.n natural language processing tasks, like text classification or language generation, it is common to use pretrained transformer models like BERT, GPT-3, or T5. These models typically have a maximum input length of 512 tokens, which means they cannot directly handle documents that exceed this limit.To process long documents with pretrained models, a common approach is to split the text into smaller segments or chunks, process each segment independently with the model, and then combine the results. By dividing the document into smaller parts, each segment can fit within the model's input length constraints.However, this approach requires careful handling to ensure that the divisions do not disrupt the coherence and context of the document. Some strategies involve using sliding windows, adding special tokens to mark the beginning and end of segments, or using an overlap between segments to preserve context.Researchers and developers often implement custom solutions to tackle long document processing, depending on their specific use case and model requirements.Long documents, like research papers, scientific articles, or books, often contain a wealth of information and insights. These documents can cover complex topics, present detailed arguments, and provide extensive evidence to support their claims. However, handling such long texts can be challenging, especially when working with models that have input length limitations.n natural language processing tasks, like text classification or language generation, it is common to use pretrained transformer models like BERT, GPT-3, or T5. These models typically have a maximum input length of 512 tokens, which means they cannot directly handle documents that exceed this limit.To process long documents with pretrained models, a common approach is to split the text into smaller segments or chunks, process each segment independently with the model, and then combine the results. By dividing the document into smaller parts, each segment can fit within the model's input length constraints.However, this approach requires careful handling to ensure that the divisions do not disrupt the coherence and context of the document. Some strategies involve using sliding windows, adding special tokens to mark the beginning and end of segments, or using an overlap between segments to preserve context.Researchers and developers often implement custom solutions to tackle long document processing, depending on their specific use case and model requirements.Long documents, like research papers, scientific articles, or books, often contain a wealth of information and insights. These documents can cover complex topics, present detailed arguments, and provide extensive evidence to support their claims. However, handling such long texts can be challenging, especially when working with models that have input length limitations.n natural language processing tasks, like text classification or language generation, it is common to use pretrained transformer models like BERT, GPT-3, or T5. These models typically have a maximum input length of 512 tokens, which means they cannot directly handle documents that exceed this limit.To process long documents with pretrained models, a common approach is to split the text into smaller segments or chunks, process each segment independently with the model, and then combine the results. By dividing the document into smaller parts, each segment can fit within the model's input length constraints.However, this approach requires careful handling to ensure that the divisions do not disrupt the coherence and context of the document. Some strategies involve using sliding windows, adding special tokens to mark the beginning and end of segments, or using an overlap between segments to preserve context.Researchers and developers often implement custom solutions to tackle long document processing, depending on their specific use case and model requirements."

In [ ]:
len(long_text)

In [1]:
import gc
import os
import sys
import threading

import numpy as np
import psutil
import torch
import random

from accelerate import Accelerator
from datasets import load_dataset
import evaluate
from torch.utils.data import DataLoader

import nltk
nltk.download("punkt")
from nltk.tokenize import sent_tokenize

from tqdm import tqdm

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, get_linear_schedule_with_warmup, set_seed

from peft import PrefixTuningConfig, TaskType, get_peft_model

accelerator = Accelerator() # device_placement="cuda:0"
model_name_or_path = "facebook/bart-base"
dataset_name = "cnn_dailymail"
peft_config = PrefixTuningConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    inference_mode=False,
    num_virtual_tokens=20,
)
text_column = 'article'
label_column = 'highlights'
lr = 3e-3
num_epochs = 10
batch_size = 32
seed = 42
do_test = True
set_seed(seed)
cnn_dataset = load_dataset(dataset_name, "3.0.0")
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
target_max_length = 128
def preprocess_function(examples):
    inputs = examples[text_column]
    targets = examples[label_column]
    model_inputs = tokenizer(inputs, truncation=True) # 这里暂时不padding
    targets = tokenizer(
        targets,
        max_length=target_max_length,
        padding='max_length',
        truncation=True,
        return_tensors='pt',
    )
    targets = targets['input_ids']
    targets[targets == tokenizer.pad_token_id] = -100
    model_inputs['labels'] = targets
    
    return model_inputs
with accelerator.main_process_first():
    cnn_dataset = cnn_dataset.map(
        preprocess_function,
        batched=True,
        num_proc=1,
        remove_columns=cnn_dataset["train"].column_names,
        load_from_cache_file=True,
        desc="Running tokenizer on dataset",
    )
accelerator.wait_for_everyone()
train_size = int(len(cnn_dataset["train"]) * 0.001)
eval_size = int(len(cnn_dataset["validation"]) * 0.001)
test_size = int(len(cnn_dataset["test"]) * 0.001)

# 从打乱后的数据集中随机抽取指定数量的数据
train_dataset = cnn_dataset["train"].shuffle(seed=42).select(range(train_size))
eval_dataset = cnn_dataset["validation"].shuffle(seed=42).select(range(eval_size))
test_dataset = cnn_dataset["test"].shuffle(seed=42).select(range(test_size))
def collate_fn(examples):
    return tokenizer.pad(examples, padding='longest', return_tensors='pt')

train_dataloader = DataLoader(
    train_dataset, 
    shuffle=True,
    collate_fn=collate_fn,
    batch_size=batch_size,
    pin_memory=True, # 将数据加载到固定的内存中，可以加速数据加载
)

for num, batch in enumerate(train_dataloader):
    print(batch["input_ids"].shape)
    break

/home/is/kaifan-l/miniconda3/envs/summarization/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to
[nltk_data]     /home/is/kaifan-l/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


torch.Size([32, 1024])


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import CrossEntropyLoss

import logging
import copy
import math
from typing import List, Optional, Tuple, Union, Iterable

from transformers import (
    BartPretrainedModel,
    BartForConditionalGeneration, 
    T5ForConditionalGeneration,
)
from transformers import BartConfig, T5Config
from transformers.modeling_outputs import Seq2SeqLMOutput

from model.prefix_encoder import PrefixEncoder
from peft import PrefixTuningConfig, TaskType, get_peft_model

logger = logging.getLogger(__name__)
# from model.summarization import BartPrefixForConditionalGeneration
checkpoint = 'facebook/bart-base'
peft_config = PrefixTuningConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    inference_mode=False,
    num_virtual_tokens=20,
)
model = BartPrefixForConditionalGeneration(    
    checkpoint=checkpoint,
    config=custom_config,
    peft_config=peft_config
)
model.model.print_trainable_parameters()

In [ ]:
for _, batch in enumerate(train_dataloader):
    input_ids = batch["input_ids"].to(model.device)
    attention_mask = batch["attention_mask"].to(model.device)
    labels = batch["labels"].to(model.device)
    
    print("input_ids.shape:", input_ids.shape)
    print("attention_mask.shape:", attention_mask.shape)
    print("labels.shape:", labels.shape)

    out = model(
        input_ids=input_ids,
        attention_mask=attention_mask,
        labels=labels,
    )
    print(out.keys())
    out = model.generate(
        input_ids=input_ids,
        # attention_mask=attention_mask,
        min_length=0,
        max_length=142,
        num_beams=4
    )
    # print("out:", tokenizer.decode(out[0], skip_special_tokens=True))
    break

In [ ]:
prefix_tokens = torch.arange(20).long()
prefix_tokens
prefix_tokens = prefix_tokens.unsqueeze(0)
prefix_tokens
prefix_tokens = prefix_tokens.expand(32, -1)
prefix_tokens

In [ ]:
from transformers import AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("facebook/bart-base")
print("pad_token_id:", tokenizer.pad_token_id)
segment_size = 20
def get_full_padding_segment():
    padding_segment = [tokenizer.pad_token_id for _ in range(segment_size)]
    return padding_segment

test = get_full_padding_segment()
test
[test]
[[12312414]] + [test] * 3

In [ ]:
ceil = math.ceil(1024/3)
ceil, 1024//3

In [ ]:
from config import PromptBartConfig
from transformers import AutoConfig
from transformers import AutoModel
from model import BartPrefixForConditionalGeneration
from peft import PrefixTuningConfig

bart_config = AutoConfig.from_pretrained('facebook/bart-base')
custom_config = PromptBartConfig(**bart_config.to_dict())
peft_config = PrefixTuningConfig.from_pretrained("kaifanli/prefix_rmt_bart_cnn-dm")
model = BartPrefixForConditionalGeneration(
    checkpoint='facebook/bart-base',
    config=custom_config,
    peft_config=peft_config,
)
model

In [ ]:
bart_config = AutoConfig.from_pretrained('facebook/bart-base')
custom_config = PromptBartConfig(**bart_config.to_dict())

In [ ]:
from transformers import AutoConfig
bart_config = AutoConfig.from_pretrained('facebook/bart-base')
custom_config = PromptBartConfig(**bart_config.to_dict())
custom_config

In [ ]:
bart_model = BartPrefixForConditionalGeneration(custom_config, tokenizer_path='facebook/bart-base')
pretrained_model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')
bart_model.model.load_state_dict(pretrained_model.state_dict())
peft_config = PrefixTuningConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    inference_mode=False,
    num_virtual_tokens=20,
)
bart_model.model = get_peft_model(bart_model.model, peft_config)
bart_model.model

In [ ]:
bart_model.model.save_pretrained('./test_peft')

In [ ]:
test_model = BartPrefixForConditionalGeneration(custom_config, tokenizer_path='facebook/bart-base')
pretrained_model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')
test_model.model.load_state_dict(pretrained_model.state_dict())
test_model

In [ ]:
from peft import PeftModel
test_model.model = PeftModel.from_pretrained(test_model.model, './test_peft')
test_model

In [ ]:
import torch

In [ ]:
padding_seg = torch.tensor([1 for _ in range(5)])
non_empyty_mask = torch.tensor([True, False, True, True, True])
padding_seg[non_empyty_mask]

In [ ]:
model_outputs = []
app = torch.tensor([1,2,3])
model_outputs.append(app)
model_outputs[0] = torch.tensor([1,2,3])
model_outputs

In [ ]:
model_outputs = [1]
not len(model_outputs)

In [ ]:
from transformers import AutoModel

In [ ]:
print('\ntest')

In [ ]:
import torch
import torch.nn.functional as F

generated_tokens = torch.tensor([[1,2,3], [4,5,6], [7,8,9]])
target_max_length = 5
generated_tokens = torch.stack([F.pad(t, (0, target_max_length - t.size(0)), value=1) for t in generated_tokens])
generated_tokens

In [ ]:
from transformers import BartForConditionalGeneration, BartTokenizer
import torch

# Load the model and tokenizer
bart_model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')

# Encode your input text into input_ids
input_text = "Because the purpose of prompt construction is to find a method that allows an LM to effectively perform a task, rather than being for human consumption, it is not necessary to limit the prompt to human-interpretable natural language. Because of this, there are also methods that examine continuous prompts (a.k.a. soft prompts) that perform prompting directly in the embedding space of the model. Specifically, continuous prompts remove two constraints: (1) relax the constraint that the embeddings of template words be the embeddings of natural language (e.g., English) words. (2) Remove the restriction that the template is parameterized by the pre-trained LM’s parameters. Instead, templates have their own parameters that can be tuned based on training data from the downstream task. We highlight several representative methods below"
input_ids = tokenizer.encode(input_text, return_tensors="pt")
print(input_ids)
# Generate output
out = bart_model.generate(input_ids, max_length=40)
print(out)
# Decode the generated output
decoded_output = tokenizer.decode(out[0], skip_special_tokens=True)
print(decoded_output)

In [1]:
from model import BartRMTForConditionalGeneration
from config import PromptBartConfig

from transformers import BartForConditionalGeneration, BartTokenizer

import torch

# Load the model and tokenizer
bart_model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')

from transformers import AutoConfig
bart_config = AutoConfig.from_pretrained('facebook/bart-base')
custom_config = PromptBartConfig(**bart_config.to_dict())

test_model = BartRMTForConditionalGeneration(custom_config, tokenizer_name_or_path='facebook/bart-base')
test_model.model.load_state_dict(bart_model.state_dict())
test_model

/home/is/kaifan-l/miniconda3/envs/summarization/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


BartRMTForConditionalGeneration(
  (model): BartForConditionalGeneration(
    (model): BartModel(
      (shared): Embedding(50265, 768, padding_idx=1)
      (encoder): BartEncoder(
        (embed_tokens): Embedding(50265, 768, padding_idx=1)
        (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
        (layers): ModuleList(
          (0-5): 6 x BartEncoderLayer(
            (self_attn): BartAttention(
              (k_proj): Linear(in_features=768, out_features=768, bias=True)
              (v_proj): Linear(in_features=768, out_features=768, bias=True)
              (q_proj): Linear(in_features=768, out_features=768, bias=True)
              (out_proj): Linear(in_features=768, out_features=768, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=768, out_features=3072, bias=True)
            (fc2): Linear(in_features=3072, out

In [3]:
test_model.model.model.encoder.layers[0].self_attn.k_proj

Linear(in_features=768, out_features=768, bias=True)

In [12]:
test = test_model.model.get_input_embeddings()
test.weight[-1]
# test(torch.tensor([[1,2,3], [4,5,6], [7,8,9]]))

tensor([-1.1010e-03, -5.4346e-02,  3.3621e-03,  2.0014e-02, -5.4740e-03,
        -2.2844e-02,  8.1586e-03,  1.0935e-02, -8.9161e-03,  8.4671e-03,
         1.2857e-02, -1.1055e-03, -1.7958e-02, -3.0604e-02,  1.3681e-02,
        -7.4673e-03,  1.4274e-03,  3.6727e-03,  3.1749e-02, -1.0873e-03,
         1.3540e-02,  8.3294e-03, -8.8273e-03,  6.1580e-03,  1.0899e-02,
        -2.4739e-03, -1.4442e-02,  4.5778e-03,  1.7585e-03, -2.3869e-03,
        -2.3175e-02,  1.7471e-02,  9.2127e-05, -7.8743e-03,  1.9892e-02,
         3.3545e-03, -2.5371e-02,  2.1744e-02, -4.6021e-03, -8.5722e-03,
        -2.3547e-02,  1.4463e-02,  4.6816e-03, -1.5214e-02,  5.4567e-03,
         1.1418e-02, -8.2172e-03,  3.1732e-03, -7.8813e-03, -8.3404e-03,
         2.2891e-02, -9.9614e-03,  6.8114e-04,  2.0809e-02,  3.7695e-02,
         2.1046e-02,  2.0017e-02, -1.2675e-02,  2.2624e-04, -1.3586e-02,
         2.3958e-02,  3.3551e-02,  3.0247e-02,  1.4323e-02, -9.8782e-03,
         6.8953e-03, -1.9203e-02,  1.9234e-02, -9.4

In [13]:
vocab_size = 50265
extended_vocab_size = vocab_size + 32
# test.resize_token_embeddings(extended_vocab_size)
test_model.model.resize_token_embeddings(extended_vocab_size)
new_embeds = test_model.model.get_input_embeddings()
new_embeds.weight[-1], new_embeds

You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embeding dimension will be 50297. This might induce some performance reduction as *Tensor Cores* will not be available. For more details  about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc


(tensor([-1.0562e-02, -6.8915e-03,  2.2760e-02, -4.8992e-03, -2.2368e-03,
          4.4224e-03, -2.3508e-02, -2.6659e-02, -2.2960e-02,  2.9333e-02,
          3.1265e-02, -3.2977e-02, -1.1224e-02, -8.8240e-03, -2.1939e-02,
          4.6327e-02, -6.8187e-03,  4.7823e-03, -1.3393e-02, -7.7753e-03,
          8.5366e-03,  9.2487e-03,  8.1838e-04,  1.0599e-02, -1.2390e-03,
         -9.9141e-03, -1.6118e-02, -1.8759e-02, -5.5478e-03,  1.6991e-02,
         -2.3425e-02, -8.6783e-03, -4.8355e-03,  2.2593e-03, -1.3003e-02,
          1.5787e-02,  1.2046e-02, -2.4905e-03,  1.7844e-02,  1.2127e-02,
          2.6637e-03, -4.1223e-03,  1.0468e-02,  2.9245e-02,  1.4475e-02,
         -9.4979e-03,  2.6671e-03,  2.7053e-02, -9.4503e-03, -3.3185e-02,
          1.5298e-02,  4.1891e-03, -1.6320e-02, -6.0812e-04, -5.6122e-03,
          2.7983e-02,  3.3466e-02, -1.4077e-02, -6.0464e-03,  6.7403e-03,
         -9.8866e-03,  2.0478e-02,  1.5812e-02, -7.5136e-03, -1.8610e-02,
          1.6465e-02,  6.7966e-04, -2.

In [4]:
input_ids = torch.tensor([[1,2,3], [4,5,6], [7,8,9]])
attention_mask = torch.tensor([[1,1,1], [1,1,1], [1,1,1]])
labels = torch.tensor([[1,2,3], [4,5,6], [7,8,9]])

test_model(
    input_ids=input_ids,
    attention_mask=attention_mask,
    labels=labels,
)

seg_kwargs['inputs_embeds'].shape=torch.Size([3, 512, 768])


Seq2SeqLMOutput(loss=tensor(6.0389, grad_fn=<MeanBackward1>), logits=tensor([[[ 0.0298,  0.0000,  5.0382,  ..., -0.7277,  0.3345, -0.7886],
         [ 0.6401,  0.0000,  4.9469,  ..., -0.7017,  0.1738, -0.9237],
         [ 0.2603,  0.0000,  4.6532,  ..., -0.4851, -0.0927, -0.4176],
         ...,
         [ 0.1754,  0.0000,  1.0521,  ..., -0.3606,  0.1263, -0.0388],
         [-0.4033,  0.0000,  0.6403,  ..., -0.2754,  0.0245, -0.9709],
         [-0.1433,  0.0000,  0.1041,  ...,  0.0193,  0.2408, -0.8375]],

        [[ 0.0608,  0.0000,  4.7712,  ..., -0.5538,  0.6644, -0.6049],
         [-0.0098,  0.0000,  4.5302,  ..., -0.7639,  0.5371,  0.2808],
         [ 0.5453,  0.0000,  4.9672,  ..., -0.4192,  0.6529,  0.0123],
         ...,
         [-0.3294,  0.0000,  0.7642,  ..., -0.2994, -0.4102,  0.3852],
         [-0.3356,  0.0000,  0.8574,  ...,  0.1801,  0.0284, -0.8613],
         [ 0.3301,  0.0000,  0.1639,  ..., -0.1535,  0.3822, -0.3297]],

        [[-0.1292,  0.0000,  5.3026,  ..., -0.2

In [5]:
for _, batch in enumerate(train_dataloader):
    input_ids = batch["input_ids"].to(test_model.device)
    attention_mask = batch["attention_mask"].to(test_model.device)
    labels = batch["labels"].to(test_model.device)
    
    print("input_ids.shape:", input_ids.shape)
    print("attention_mask.shape:", attention_mask.shape)
    print("labels.shape:", labels.shape)

    out = test_model(
        input_ids=input_ids,
        attention_mask=attention_mask,
        labels=labels,
    )
    print(out.keys())
    out = test_model.generate(
        input_ids=input_ids,
        # attention_mask=attention_mask,
        min_length=0,
        max_length=142,
        num_beams=4
    )

input_ids.shape: torch.Size([32, 1024])
attention_mask.shape: torch.Size([32, 1024])
labels.shape: torch.Size([32, 128])
seg_kwargs['inputs_embeds'].shape=torch.Size([32, 512, 768])
seg_kwargs['inputs_embeds'].shape=torch.Size([32, 512, 768])
memory.shape=torch.Size([32, 20, 768])
seg_kwargs['inputs_embeds'][:, self.pre_seq_len, :].shape=torch.Size([32, 768])
seg_kwargs['inputs_embeds'].shape=torch.Size([32, 512, 768])
memory.shape=torch.Size([32, 20, 768])
seg_kwargs['inputs_embeds'][:, self.pre_seq_len, :].shape=torch.Size([32, 768])
seg_kwargs['inputs_embeds'].shape=torch.Size([32, 512, 768])
seg_kwargs['inputs_embeds'].shape=torch.Size([32, 512, 768])
seg_kwargs['inputs_embeds'].shape=torch.Size([32, 512, 768])
input_ids.shape: torch.Size([32, 1024])
attention_mask.shape: torch.Size([32, 1024])
labels.shape: torch.Size([32, 128])
seg_kwargs['inputs_embeds'].shape=torch.Size([32, 512, 768])
seg_kwargs['inputs_embeds'].shape=torch.Size([32, 512, 768])
memory.shape=torch.Size([32, 20,

In [10]:
from transformers.activations import ACT2FN

In [16]:
from transformers.activations import ACT2FN
from transformers.modeling_outputs import (
    BaseModelOutput,
    BaseModelOutputWithPastAndCrossAttentions,
    CausalLMOutputWithCrossAttentions,
    Seq2SeqLMOutput,
    Seq2SeqModelOutput,
    Seq2SeqQuestionAnsweringModelOutput,
    Seq2SeqSequenceClassifierOutput,
)
from transformers.modeling_utils import PreTrainedModel
from transformers.utils import (
    add_code_sample_docstrings,
    add_end_docstrings,
    add_start_docstrings,
    add_start_docstrings_to_model_forward,
    logging,
    replace_return_docstrings,
)
from transformers import BartConfig

In [4]:
split_inds = (list(range(127, 0, -16)) + [0])[::-1]
split_inds

[0, 15, 31, 47, 63, 79, 95, 111, 127]

In [11]:
split_inds = (list(range(126, 0, -16)) + [0])[::-1]
split_inds

[0, 14, 30, 46, 62, 78, 94, 110, 126]

In [17]:
import torch
a = torch.arange(1,1)
len(a) == 0

True

In [1]:
from utils import SummarizationMetric

In [2]:
sum_metric = SummarizationMetric()

In [ ]:
import evaluate
import nltk
nltk.download("punkt")
from nltk.tokenize import sent_tokenize
from tqdm import tqdm
import torch.nn.functional as F
from .trace_malloc import * 


rouge = evaluate.load('rouge')
predictions = ["Photographer Jeffrey Milstein has shot a fascinating project documenting black boxes, shedding light on what happens to the devices after tragic crashes.\nDespite the name, the boxes are usually red to help rescuers spot them among the wreckage of crashed planes and helicopters.\nMr Milstein's work saw him gain access to air safety storage facilities where data recorders end up following an accident.", "Cruise ship passengers on today relived their two day ordeal at sea after being trapped on Carnival Spirit as Sydney was battered by the storm of the century.\nHuge waves as high as 40 feet smashed glass panels, shattered plates and cups – and left many on-board violently seasick as the boat pitched from side to side.\nA Sydney harbour pilot finally boarded this morning and the ship, carrying 2,500 holidaymakers and 1,500 crew, docked at the Overseas Passenger Terminal at around 10am.\n'Inside everyone was sick, it was really quite rocky,' one passenger said.", 'Fred Hatch, 76, was battered to death with a claw hammer by his neighbour.\nHis wife Enid, 70, ran out to help what she thought was an injured man.\nShe discovered blood splattered walls where her husband had been beaten.\nThe man was in fact Alan Rogers, 73, who had smashed the skull.\nRogers was sentenced to a hospital order under the Mental Health Act.\nHe may never be released and was suffering from paranoid schizophrenia.', 'Floyd Mayweather v Manny Pacquiao will be the biggest fight of all time financially and the most significant this century.\nThe perceived wisdom was that Ali-Frazier was coming five years too late for two great boxers in their 30s.\nLet us hope they are so spectacularly wrong again.\nIn this, the fifth in a series of 12 fights that shaped boxing history, I look back on one of the greatest sporting events.', "TripAdvisor names Providenciales as the world's top island.\nAmbergris Caye had held the top ranking for two years in a row.\nKo Tao, Thailand, gained five spots this year, moving up from No.\n10 last year.\nHere are the top 10 U.S. islands:", 'Michael Scott Shemansky is on the run after authorities named him as a suspect in the murder of his mother.\nThe body was discovered at the West Colonial Drive home when an out-of-state family member called in a well-being check.\nHe is already on parole for battery on law enforcement, has a history of bipolar mental illness, and was involuntarily committed to a psychiatric ward for an attempted suicide.\nHis wife filed a domestic violence injunction against him two years ago, according to her attorney.', 'Dame Sally Davies is said to be concerned at number of children suffering from the condition, which is caused by a deficiency in vitamin D. The disease, a scourge of Victorian Britain, was virtually eradicated after the Second World War.\nBut it is returning as more and more youngsters are used to staying indoors playing video games than going outside.\nNow, it has been reported that Professor Davies has ordered the National Institute for Health and Clinical Excellence to review the cost of providing vitamin supplements.', 'John Doyle, 79, was helping two 12-year-old girls cross the road to school.\nGenti Rustemi, 45, drove over the crossing point without slowing down.\nMr Doyle tapped his lollipop stick on the roof of the BMW as it hurtled past.\nHe then told the driver to be more careful and told him to open his eyes.\nBut after Rustemi had dropped off his daughter at a different school, he returned to the crossing minutes later in a fit of temper and attacked Mr Doyle.']

references = ['Fascinating photographs show black boxes salvaged from the wreckage of plane and helicopter crashes.\nJeffrey Milstein took the pictures to shed light on what happens to flight recorders after tragic accidents.\nMany of them are dented, mangled and even damaged by fire but they could still be used by crash investigators.', '4,000 on board the Carnival Spirit have been stranded outside Sydney Harbour.\nPassengers were to disembark the ship on Tuesday but the wild storm prevented it from entering the harbour.\nThe vessel has suffered damage with smashed glass panels and a door ripped open by the crashing waves.\nCarnival Spirit was returning from a 12-night cruise to New Caledonia, Vanuatu and Fiji.', "Enid Hatch found blood splattered walls after going to look for her husband.\nHe had been beaten with a claw hammer by a crazed pensioner neighbour.\nAlan Rogers sentenced under Mental Health Act and may never be released.\nMrs Hatch's sister Betty was murdered in 1971 by mental health absconder.", "Floyd Mayweather v Manny Pacquiao is now just eight days away.\nSportsmail's Jeff Powell has been counting down the greatest fights.\nIn the fifth in a series of 12 fights that shaped boxing history, we have Muhammad Ali v Joe Frazier, the Thrilla in Manila.\nIt was said to have come\xa0five years late, much like Mayweather v Pacquiao.\nYet those doubting the two greats in their 30s were spectacularly wrong.", "Providenciales in the Turks and Caicos is the top TripAdvisor Travelers' Choice award winner for islands.\nMaui ranks first on the top 10 islands list for the United States.", 'Police say Michael Scott Shemansky came to their attention after he failed to appear for a supervised visit with his son Saturday.\nThat same day mother Sandra Shemansky, 57, was found dead at the home they shared in Winter Garden, Florida.\nMichael Shemansky was going through a difficult divorce and neighbors believe the stress may have caused him to snap.', 'Dame Sally Davies has ordered review into cost of giving out free vitamins.\nComes after a rise in the number of cases of rickets in children in the UK.\nIncrease is being put down to fact children spend less time outside playing.\nRickets can cause bone deformities such as bowed legs and a curved spine.', "Parent Genti Rustemi 45, dropping off daughter and refused to slow down.\nHe took child to school, then returned and punched lollipop man John Doyle to the floor in'repugnant' attack in front of schoolchildren.\nChip shop bosses admitted attack and given suspended sentence.\nGrandfather says Rustemi's failure to stop was 'like going through a red light', and both he and the children\xa0'could have been run over'"]

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # ROUGE expects a newline after each sentence
    preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

    return preds, labels
predictions, references=postprocess_text(predictions, references)
print(predictions[0])
results = rouge.compute(predictions=predictions,
                        references=references,
                        rouge_types=['rouge1', 'rouge2', 'rougeL', 'rougeLsum'],
                        use_aggregator=True)
print(list(results.keys()))
# ['rouge1', 'rouge2', 'rougeL', 'rougeLsum']
print(results)
# [0.5, 0.0]
# use_aggregator = False时，输出每个句子的评分

# use_aggregator = True时，得分求平均，输出为一个值
print(results["rouge1"])
0.25


In [19]:
print('123\n123')

123
123


In [5]:
model = BartRMTForConditionalGeneration.from_pretrained('kaifanli/RMTbart-base', tokenizer_name_or_path='facebook/bart-base')

Some weights of BartRMTForConditionalGeneration were not initialized from the model checkpoint at kaifanli/RMTbart-base and are newly initialized: ['model.model.encoder.layers.0.self_attn.k_proj.weight', 'model.model.decoder.layers.2.self_attn.q_proj.weight', 'model.model.encoder.layers.4.self_attn.v_proj.weight', 'model.model.encoder.layers.5.self_attn.v_proj.weight', 'model.model.encoder.layers.1.self_attn.q_proj.weight', 'model.model.encoder.layernorm_embedding.bias', 'model.model.decoder.layers.0.encoder_attn.k_proj.weight', 'model.model.encoder.layers.2.self_attn.out_proj.weight', 'model.model.decoder.layers.5.fc2.bias', 'model.model.encoder.layers.5.self_attn.k_proj.weight', 'model.model.decoder.layers.2.self_attn.out_proj.bias', 'model.model.decoder.layers.3.self_attn.k_proj.bias', 'model.model.decoder.layers.2.final_layer_norm.bias', 'model.model.encoder.layers.1.self_attn.v_proj.bias', 'model.model.decoder.layers.3.encoder_attn_layer_norm.bias', 'model.model.encoder.layers.1.f

In [6]:
model


BartRMTForConditionalGeneration(
  (model): BartForConditionalGeneration(
    (model): BartModel(
      (shared): Embedding(50281, 768, padding_idx=1)
      (encoder): BartEncoder(
        (embed_tokens): Embedding(50281, 768, padding_idx=1)
        (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
        (layers): ModuleList(
          (0-5): 6 x BartEncoderLayer(
            (self_attn): BartAttention(
              (k_proj): Linear(in_features=768, out_features=768, bias=True)
              (v_proj): Linear(in_features=768, out_features=768, bias=True)
              (q_proj): Linear(in_features=768, out_features=768, bias=True)
              (out_proj): Linear(in_features=768, out_features=768, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=768, out_features=3072, bias=True)
            (fc2): Linear(in_features=3072, out

In [7]:
test_model = BartForConditionalGeneration.from_pretrained('kaifanli/RMTbart-base')


Some weights of the model checkpoint at kaifanli/RMTbart-base were not used when initializing BartForConditionalGeneration: ['embeddings.weight']
- This IS expected if you are initializing BartForConditionalGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BartForConditionalGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
test_model

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50281, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50281, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=

In [9]:
face = BartForConditionalGeneration.from_pretrained('facebook/bart-base')
face

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=

In [ ]:
from transformers.configuration_utils import 